In [39]:
import qiskit as qs
from qiskit_aer import Aer
from qiskit import QuantumCircuit, QuantumRegister,ClassicalRegister

regs = [QuantumRegister(2, 'q'), ClassicalRegister(1, 'c')]
# State initialization: apply x on output bit, and H on all
init = QuantumCircuit(*regs)
init.x(1)
init.h(0)
init.h(1)
init.barrier() # Only for separation in printing

CircuitInstruction(operation=Instruction(name='barrier', num_qubits=2, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(2, 'q'), 0), Qubit(QuantumRegister(2, 'q'), 1)), clbits=())

In [25]:
print(init)

     ┌───┐      ░ 
q_0: ┤ H ├──────░─
     ├───┤┌───┐ ░ 
q_1: ┤ X ├┤ H ├─░─
     └───┘└───┘ ░ 
c: 1/═════════════
                  


In [26]:
# Apply h to input bit and measure it at the end
end = QuantumCircuit(*regs)
end.h(0)
end.measure(0, 0)

In [27]:
print(end)

     ┌───┐┌─┐
q_0: ┤ H ├┤M├
     └───┘└╥┘
q_1: ──────╫─
           ║ 
c: 1/══════╩═
           0 


In [28]:
print(init & end)

     ┌───┐      ░ ┌───┐┌─┐
q_0: ┤ H ├──────░─┤ H ├┤M├
     ├───┤┌───┐ ░ └───┘└╥┘
q_1: ┤ X ├┤ H ├─░───────╫─
     └───┘└───┘ ░       ║ 
c: 1/═══════════════════╩═
                        0 


In [29]:
# Unbalanced functions
# unbalanced0: will simply be an identity (the unity)
unbalanced0 = QuantumCircuit(*regs)
unbalanced0.barrier()

CircuitInstruction(operation=Instruction(name='barrier', num_qubits=2, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(2, 'q'), 0), Qubit(QuantumRegister(2, 'q'), 1)), clbits=())

In [31]:
# unbalanced1: apply NOT to the y registrer - output will flip independent of input
unbalanced1 = QuantumCircuit(*regs)
unbalanced1.x(1)
unbalanced1.barrier()

CircuitInstruction(operation=Instruction(name='barrier', num_qubits=2, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(2, 'q'), 0), Qubit(QuantumRegister(2, 'q'), 1)), clbits=())

In [32]:
print(init & unbalanced0 & end)

     ┌───┐      ░  ░ ┌───┐┌─┐
q_0: ┤ H ├──────░──░─┤ H ├┤M├
     ├───┤┌───┐ ░  ░ └───┘└╥┘
q_1: ┤ X ├┤ H ├─░──░───────╫─
     └───┘└───┘ ░  ░       ║ 
c: 1/══════════════════════╩═
                           0 


In [33]:
print(init & unbalanced1 & end)

     ┌───┐      ░       ░ ┌───┐┌─┐
q_0: ┤ H ├──────░───────░─┤ H ├┤M├
     ├───┤┌───┐ ░ ┌───┐ ░ └───┘└╥┘
q_1: ┤ X ├┤ H ├─░─┤ X ├─░───────╫─
     └───┘└───┘ ░ └───┘ ░       ║ 
c: 1/═══════════════════════════╩═
                                0 


In [47]:
# Balanced function: C-NOT - output depends on input
balanced = QuantumCircuit(*regs)
balanced.cx(0,1)
balanced.barrier()

CircuitInstruction(operation=Instruction(name='barrier', num_qubits=2, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(2, 'q'), 0), Qubit(QuantumRegister(2, 'q'), 1)), clbits=())

In [35]:
print(init & balanced & end)

     ┌───┐      ░       ░ ┌───┐┌─┐
q_0: ┤ H ├──────░───■───░─┤ H ├┤M├
     ├───┤┌───┐ ░ ┌─┴─┐ ░ └───┘└╥┘
q_1: ┤ X ├┤ H ├─░─┤ X ├─░───────╫─
     └───┘└───┘ ░ └───┘ ░       ║ 
c: 1/═══════════════════════════╩═
                                0 


In [41]:
# Transpile the circuit, feed it in to the simulator and extract the results
sim = Aer.get_backend('qasm_simulator')

# Expected: only 1s for balanced, only 0s for unbalanced
for kind, oracle in (('b', balanced),
                     ('u', unbalanced0),
                     ('u', unbalanced1)):
    circuit = init & oracle & end
    transpiled = qs.transpile(circuit)
    counts = sim.run(transpiled, shots=10).result().get_counts()
    print(kind, counts)

b {'1': 10}
u {'0': 10}
u {'0': 10}


In [48]:
import cirq
# Create the qubits 
qubits = [cirq.LineQubit(i) for i in (0, 1)]

# Create a circuit
init = cirq.Circuit(
    cirq.H(qubits[0]),
    cirq.X(qubits[1]),
    cirq.H(qubits[1])
)
end = cirq.Circuit(
    cirq.H(qubits[0]),
    cirq.measure(qubits[0], key='m')
)

In [49]:
print(init)

0: ───H───────

1: ───X───H───


In [50]:
print(end)

0: ───H───M('m')───


In [52]:
print(init + end)

0: ───H───────H───M('m')───

1: ───X───H────────────────


In [53]:
# Write function circuits
balanced = cirq.Circuit(
    cirq.CNOT(*qubits)
)   
    
unbalanced0 = cirq.Circuit()
unbalanced1 = cirq.Circuit(
    cirq.X(qubits[1])
)

In [54]:
# Simulate the circuit several times.
simulator = cirq.Simulator()
# expected: only 1s for balanced, only 0s for unbalanced
for kind, oracle in (('b', balanced), 
                     ('u', unbalanced0),
                     ('u', unbalanced1)): 
    result = simulator.run(init + oracle + end, repetitions=10) 
    print(kind, result) 

b m=1111111111
u m=0000000000
u m=0000000000
